In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv("kc_house_data.csv")
df = df.drop(columns=["id", "date", "zipcode"])
df["price"] = df["price"] / 1000
X = df.drop(columns=["price"])
y = df["price"].values.reshape(-1, 1)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
def add_intercept(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

X_train_b = add_intercept(X_train_scaled)
X_test_b = add_intercept(X_test_scaled)

In [5]:
def gradient_descent(X, y, alpha, num_iters):
    n, d = X.shape
    theta = np.zeros((d, 1))

    for _ in range(num_iters):
        y_pred = X @ theta
        grad = (2 / n) * (X.T @ (y_pred - y))
        theta = theta - alpha * grad
        
    return theta

In [6]:
def predict(X, theta):
    return X @ theta

In [7]:
alphas = [0.01, 0.1, 0.5]
iters_list = [10, 50, 100]

rows = []

for alpha in alphas:
    for iters in iters_list:
        theta = gradient_descent(X_train_b, y_train, alpha, iters)
        y_pred_train = predict(X_train_b, theta).ravel()
        y_pred_test = predict(X_test_b, theta).ravel()
        
        mse_train = mean_squared_error(y_train.ravel(), y_pred_train)
        r2_train = r2_score(y_train.ravel(), y_pred_train)
        mse_test = mean_squared_error(y_test.ravel(), y_pred_test)
        r2_test = r2_score(y_test.ravel(), y_pred_test)
        theta_str = np.array2string(theta.ravel(), precision=6, separator=", ")
        
        rows.append([alpha, iters, theta_str, mse_train, r2_train, mse_test, r2_test])

In [8]:
results_gd = pd.DataFrame(
    rows,
    columns=["alpha", "iterations", "theta", "MSE_train", "R2_train", "MSE_test", "R2_test"]
)

results_gd

,alpha,iterations,theta,MSE_train,R2_train,MSE_test,R2_test
0,0.01,10,"[98.3724 , 12.686749, 24.242477, 34.585526, ...",2.595410e+05,-9.865578e-01,2.775394e+05,-8.358607e-01
1,0.01,50,"[341.92923 , 3.559408, 33.343614, 62.39491...",8.172058e+04,3.745010e-01,9.051283e+04,4.012780e-01
2,0.01,100,"[466.449488, -11.152577, 31.007245, 68.79908...",4.600254e+04,6.478911e-01,5.304284e+04,6.491335e-01
3,0.10,10,"[480.025643, -12.459975, 30.62795 , 69.19147...",4.414163e+04,6.621347e-01,5.102153e+04,6.625040e-01
4,0.10,50,"[537.760373, -30.058615, 33.66238 , 79.02538...",3.984712e+04,6.950054e-01,4.599467e+04,6.957556e-01
5,0.10,100,"[537.768048, -30.565816, 34.75278 , 79.37310...",3.983439e+04,6.951029e-01,4.599601e+04,6.957468e-01
6,0.50,10,"[ 5.377680e+02, -3.373430e+07, -5.018860e+07, ...",9.263391e+16,-7.090310e+11,1.001676e+17,-6.625861e+11
7,0.50,50,"[-1.225899e+20, -1.026138e+32, -1.526653e+32, ...",8.571168e+65,-6.560474e+60,9.268238e+65,-6.130731e+60
8,0.50,100,"[-4.924761e+50, -4.122158e+62, -6.132803e+62, ...",1.383176e+127,-1.058699e+122,1.495666e+127,-9.893496e+121
